In [ ]:
import torch
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
import time
import copy
import matplotlib.pyplot as plt
import os
import random

seed=42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Normalisation des images pour les modèles pré-entraînés PyTorch

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
image_directory = "/content/drive/MyDrive/ProjetFromage/data_fromage"
dataset_full = datasets.ImageFolder(image_directory, data_transforms)

# split en train, val et test à partir de la liste complète
np.random.seed(42)
samples_train, samples_test = train_test_split(dataset_full.samples)
samples_train, samples_val = train_test_split(samples_train,test_size=0.2)

print("Nombre d'images de train : %i" % len(samples_train))
print("Nombre d'images de val : %i" % len(samples_val))
print("Nombre d'images de test : %i" % len(samples_test))

Nombre d'images de train : 2288
Nombre d'images de val : 572
Nombre d'images de test : 954


In [ ]:
# on définit les datasets et loaders pytorch à partir des listes d'images de train / val / test
dataset_train = datasets.ImageFolder(image_directory, data_transforms)
dataset_train.samples = samples_train
dataset_train.imgs = samples_train

dataset_val = datasets.ImageFolder(image_directory, data_transforms)
dataset_val.samples = samples_val
dataset_val.imgs = samples_val

dataset_test = datasets.ImageFolder(image_directory, data_transforms)
dataset_test.samples = samples_test
dataset_test.imgs = samples_test

torch.manual_seed(42)

In [ ]:
# détermination du nombre de classes
# vérification que les labels sont bien dans [0, nb_classes]
labels=[x[1] for x in samples_train]
if np.min(labels) != 0:
    print("Error: labels should start at 0 (min is %i)" % np.min(labels))
    sys.exit(-1)
if np.max(labels) != (len(np.unique(labels))-1):
    print("Error: labels should go from 0 to Nclasses (max label = {}; Nclasse = {})".format(np.max(labels),len(np.unique(labels)))  )
    sys.exit(-1)
nb_classes = np.max(labels)+1
print("Apprentissage sur {} classes".format(nb_classes))

Apprentissage sur 43 classes


In [ ]:
# on définit le device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [ ]:
# on définit une fonction d'évaluation
def evaluate(model, dataset):
    avg_loss = 0.
    avg_accuracy = 0
    loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False, num_workers=2)
    for data in loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        n_correct = torch.sum(preds == labels)
        
        avg_loss += loss.item()
        avg_accuracy += n_correct
        
    return avg_loss / len(dataset), float(avg_accuracy) / len(dataset)

# fonction d'entraînement du modèle
PRINT_LOSS = False

def train_model(model, loader_train, data_val, optimizer, criterion, n_epochs=10):
  
    since = time.time()
    best_acc=0
    best_model_wts = copy.deepcopy(model.state_dict())
    saved_acc=[]

    for epoch in range(n_epochs): 
        batch=0
        print("EPOCH % i" % epoch)
        for i, data in enumerate(loader_train):
            batch+=1
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device) 
            optimizer.zero_grad() 
            outputs = model(inputs) 
            
            loss = criterion(outputs, labels) 
            if PRINT_LOSS:
                model.train(False)
                loss_val, accuracy = evaluate(model, data_val)
                model.train(True)
                print("{} loss train: {:1.4f}\t val {:1.4f}\tAcc (val): {:.1%}".format(i, loss.item(), loss_val, accuracy   ))
            
            loss.backward() 
            optimizer.step() 
            #print(f'Batch {batch} : done')

        model.train(False)
        loss_val, accuracy = evaluate(model, data_val)
        saved_acc.append(accuracy)
        #early stopping
        if accuracy>best_acc:
          best_acc=accuracy
          best_model_wts = copy.deepcopy(model.state_dict())
        model.train(True)
        print("{} loss train: {:1.4f}\t val {:1.4f}\tAcc (val): {:.1%}".format(i, loss.item(), loss_val, accuracy   ))

    model.load_state_dict(best_model_wts)
    return best_acc


In [ ]:
#===== Transfer learning "simple" (sans fine tuning) =====

global_best_acc=0
best_param={}
times={}
best_net=models.resnet18(pretrained=True)

for batch_size in [8, 16, 32]:
  # Récupération du réseau pré-entraîné (resnet-18)
  my_net = models.resnet18(pretrained=True)
  loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=2)

  for param in my_net.parameters():
      param.requires_grad = False

  my_net.fc = nn.Linear(in_features=my_net.fc.in_features, out_features=nb_classes, bias=True)
  my_net.to(device) 
  my_net.train(True) 

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(my_net.fc.parameters(), lr=0.001, momentum=0.9)

  print(f'Apprentissage en transfer learning : Batch size = {batch_size}')
  start = time.time()
  my_net.train(True)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  best_acc=train_model(my_net, loader_train, dataset_val, optimizer, criterion, n_epochs=10)
  times[batch_size]=time.time()-start
  if best_acc>global_best_acc:
    global_best_acc=best_acc
    best_param={"Batch size":batch_size}
    best_net=my_net

print(">>> Best param :")
for key, item in best_param.items():
  print(f'{key} = {item}')

# évaluation
best_net.train(False)
loss, accuracy = evaluate(best_net, dataset_test)
print("Accuracy (test): %.1f%%" % (100 * accuracy))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Apprentissage en transfer learning : Batch size = 8
EPOCH  0
285 loss train: 2.5596	 val 0.1732	Acc (val): 31.8%
EPOCH  1
285 loss train: 2.4215	 val 0.1399	Acc (val): 45.3%
EPOCH  2
285 loss train: 1.6320	 val 0.1249	Acc (val): 49.0%
EPOCH  3
285 loss train: 1.3557	 val 0.1145	Acc (val): 50.3%
EPOCH  4
285 loss train: 1.6579	 val 0.1077	Acc (val): 53.5%
EPOCH  5
285 loss train: 1.5273	 val 0.1056	Acc (val): 55.9%
EPOCH  6
285 loss train: 1.1088	 val 0.1031	Acc (val): 56.5%
EPOCH  7
285 loss train: 0.9767	 val 0.0981	Acc (val): 58.9%
EPOCH  8
285 loss train: 0.9427	 val 0.0960	Acc (val): 59.6%
EPOCH  9
285 loss train: 1.4079	 val 0.1000	Acc (val): 58.2%
Apprentissage en transfer learning : Batch size = 16
EPOCH  0
142 loss train: 3.1576	 val 0.1979	Acc (val): 22.6%
EPOCH  1
142 loss train: 2.9054	 val 0.1672	Acc (val): 38.1%
EPOCH  2
142 loss train: 2.2691	 val 0.1474	Acc (val): 43.5%
EPOCH  3
142 loss train: 1.5536	 val 0.1331	Acc (val): 48.6%
EPOCH  4
142 loss train: 1.9337	 val 0.12

In [ ]:
torch.cuda.empty_cache()

#===== Fine tuning =====

global_best_acc=0
best_param={}
times_ft={}
best_net_ft=models.resnet18(pretrained=True)

for batch_size in [8,16,32]:
  # on réinitialise resnet
  my_net_ft = models.resnet18(pretrained=True)
  my_net_ft.fc = nn.Linear(in_features=my_net_ft.fc.in_features, out_features=nb_classes, bias=True)
  my_net_ft.to(device)

  loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=1)

  # cette fois on veut updater tous les paramètres
  params_to_update = my_net_ft.parameters()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

  # on ré-entraîne
  print(f'Apprentissage avec fine tuning : Batch size = {batch_size}')
  start=time.time()
  my_net_ft.train(True)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  best_acc=train_model(my_net_ft, loader_train, dataset_val, optimizer, criterion, n_epochs=10)
  times_ft[batch_size]=time.time()-start
  if best_acc>global_best_acc:
    global_best_acc=best_acc
    best_param={"Batch size":batch_size}
    best_net_ft=my_net_ft

print(">>> Best param :")
for key, item in best_param.items():
  print(f'{key} = {item}')

# on ré-évalue les performances
best_net_ft.train(False)
loss, accuracy = evaluate(best_net_ft, dataset_test)
print("Accuracy (test): %.1f%%" % (100 * accuracy))

Apprentissage avec fine tuning : Batch size = 8
EPOCH  0
285 loss train: 1.0891	 val 0.1062	Acc (val): 50.2%
EPOCH  1
285 loss train: 1.6860	 val 0.0755	Acc (val): 69.4%
EPOCH  2
285 loss train: 0.4629	 val 0.0629	Acc (val): 73.1%
EPOCH  3
285 loss train: 0.3120	 val 0.0564	Acc (val): 75.7%
EPOCH  4
285 loss train: 0.1758	 val 0.0545	Acc (val): 76.4%
EPOCH  5
285 loss train: 0.0806	 val 0.0568	Acc (val): 75.3%
EPOCH  6
285 loss train: 0.0176	 val 0.0542	Acc (val): 76.4%
EPOCH  7
285 loss train: 0.0206	 val 0.0551	Acc (val): 76.2%
EPOCH  8
285 loss train: 0.0190	 val 0.0567	Acc (val): 77.8%
EPOCH  9
285 loss train: 0.0156	 val 0.0583	Acc (val): 75.9%
Apprentissage avec fine tuning : Batch size = 16
EPOCH  0
142 loss train: 2.2740	 val 0.1374	Acc (val): 46.3%
EPOCH  1
142 loss train: 1.9135	 val 0.0956	Acc (val): 60.3%
EPOCH  2
142 loss train: 0.9981	 val 0.0782	Acc (val): 70.8%
EPOCH  3
142 loss train: 0.5848	 val 0.0666	Acc (val): 72.2%
EPOCH  4
142 loss train: 0.3907	 val 0.0587	Acc (